In [9]:
import numpy as np
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def generate_paths(S0, r, sigma, T, M, N):
    """
    Generate stock price paths using geometric Brownian motion.

    Args:
        S0 (float): Initial stock price.
        r (float): Risk-free interest rate.
        sigma (float): Volatility.
        T (float): Time to maturity.
        M (int): Number of time steps.
        N (int): Number of paths.

    Returns:
        numpy.ndarray: Array of shape (M+1, N) representing the stock price paths.
    """
    dt = T / M
    S = np.zeros((M + 1, N))
    S[0] = S0
    for t in range(1, M + 1):
        z = np.random.standard_normal(N)
        S[t] = S[t - 1] * np.exp((r - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * z)
    return S

def LSMC_American_Option(S0, K, r, sigma, T, M, N):
    # Function body with appropriate indentation
    dt = T / M
    S = generate_paths(S0, r, sigma, T, M, N)
    V = np.maximum(K - S[-1], 0)  # Payoff at maturity

    for t in range(M - 1, 0, -1):
        regression = np.polyfit(S[t], V * np.exp(-r * dt), 3)
        C = np.polyval(regression, S[t])  # Continuation values
        V = np.where(K - S[t] > C, K - S[t], V * np.exp(-r * dt))

    V0 = np.exp(-r * dt) * np.mean(V)
    return V0

def price_american_option(S0, K, r, sigma, T, M, N):
    try:
        option_price = LSMC_American_Option(S0, K, r, sigma, T, M, N)
        logger.info("American Option Price: %f", option_price)
        return option_price
    except Exception as e:
        logger.exception("Error occurred while pricing American option: %s", str(e))
        raise

if __name__ == "__main__":
    S0 = 100
    K = 100
    r = 0.05
    sigma = 0.2
    T = 1.0
    M = 100
    N = 10000

    price_american_option(S0, K, r, sigma, T, M, N)

INFO:__main__:American Option Price: 5.967034
